In [1]:
import io
from PIL import Image
import torch
import os
import torchvision.transforms as transforms 
import torch.nn as nn
import timm
from torchvision.transforms import ToTensor
from torchvision.transforms import v2
import numpy as np
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

with open('animal names.txt') as f:
    class_names = [x.strip('\n') for x in f.readlines()]

c:\SAKET\Coding\PYTHON\Python Projects\Python Deep Learning Projects\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


In [2]:
model = timm.create_model('resnet50',
                          pretrained=True,
                          num_classes = 90).to(device)

In [3]:
MODEL_SAVE_PATH = '../pytorch saved model/AnimalImageClassificationResnet50.pth'
loaded_model = model
loaded_model.load_state_dict(torch.load(f=MODEL_SAVE_PATH ,map_location=torch.device('cpu')))
loaded_model = loaded_model.to(device)

In [4]:
loaded_model.eval()
stats = ((0.5087, 0.5006, 0.4405),(0.2128, 0.2084, 0.2120))

def transform_image(image_bytes):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(size=(224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize(*stats,inplace = True),
        ])

    image = Image.open(io.BytesIO(image_bytes))
    return transform(image)

In [9]:
files={'file': open('images/test4.webp', 'rb')}
file = files['file']
image_byte = file.read()
import matplotlib.pyplot as plt
tensor = transform_image(image_byte)

In [10]:
def get_prediction(image_tensor):
    images = image_tensor.unsqueeze(0)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    return f'Model Predicted Image as : {class_names[predicted]}'

In [12]:
# get_prediction(tensor)